In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np


# Анотування тексту за допомогою `nltk`

In [2]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to C:\Users\Sergiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sergiy/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

## Створюємо теку з українськими стоп-словами

In [3]:
import os
import requests

# Перевіряємо наявність папки corpora/stopwords у каталозі nltk_data
nltk_data_path = nltk.data.path[0]
stopwords_path = os.path.join(nltk_data_path, "corpora", "stopwords")
if not os.path.exists(stopwords_path):
    os.makedirs(stopwords_path)

# Загружаем стоп-слова для украинского языка и сохраняем в файл
url = (
    "https://raw.githubusercontent.com/olegdubetcky/Ukrainian-Stopwords/main/ukrainian"
)
r = requests.get(url)

with open(os.path.join(stopwords_path, "ukrainian"), "wb") as f:
    f.write(r.content)

# Добавляем украинские стоп-слова в NLTK
stop_words = set(stopwords.words("ukrainian"))
with open(os.path.join(stopwords_path, "ukrainian"), "r", encoding="utf-8") as f:
    custom_stopwords = set(f.read().splitlines())

# Объединяем стандартные и кастомные стоп-слова
stop_words.update(custom_stopwords)

## Завантажуємо текст з файлу у змінну

In [4]:
with open("text_ua.txt", "r", encoding="utf-8") as file:
    text = file.read()


## Визначаємо функції

In [5]:
def read_text(text):
    """
    Функція використовується для поділу тексту на речення за допомогою nltk.sent_tokenize.
    """
    sentences = sent_tokenize(text)
    return sentences


def sentence_similarity(sent1, sent2, stopwords=None):
    """
    Функція вимірює схожість між двома реченнями. Використовується косинусна відстань між векторами, що представляють слова в реченнях. Вона також враховує стоп-слова.
    """
    if stopwords is None:
        stopwords = set()

    words1 = [
        word.lower()
        for word in word_tokenize(sent1)
        if word.isalnum() and word.lower() not in stopwords
    ]
    words2 = [
        word.lower()
        for word in word_tokenize(sent2)
        if word.isalnum() and word.lower() not in stopwords
    ]

    all_words = list(set(words1 + words2))

    vector1 = [1 if word in words1 else 0 for word in all_words]
    vector2 = [1 if word in words2 else 0 for word in all_words]

    return 1 - cosine_distance(vector1, vector2)


def build_similarity_matrix(sentences, stop_words):
    """
    Функція будує матрицю схожості між усіма парами речень у тексті.
    """
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity_matrix[i][j] = sentence_similarity(
                    sentences[i], sentences[j], stop_words
                )

    return similarity_matrix


def generate_summary(text, num_sentences=3):
    """
    Функція створює короткий зміст тексту.

    Алгоритм роботи:

    1. Читаємо речення із тексту і потім будуємо матрицю схожості.
    2. Розраховуємо суму схожості для кожного речення.
    3. Речення сортуються в порядку спадання цієї суми.
    4. Вибибираємо топ N речень (де N - кількість речень у стислому змісті) за порядком зменшення схожості й об'єднуємо їх в один рядок.

    """
    stop_words = set(stopwords.words("ukrainian"))
    summarize_text = []

    sentences = read_text(text)

    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    sentence_similarity_scores = np.array(
        [sum(row) for row in sentence_similarity_matrix]
    )

    ranked_sentences = [
        sentence
        for _, sentence in sorted(
            zip(sentence_similarity_scores, sentences), reverse=True
        )
    ]

    for i in range(min(num_sentences, len(ranked_sentences))):
        summarize_text.append(ranked_sentences[i])

    return " ".join(summarize_text)


## Записуємо анотований текст до файлу

In [6]:
summary = generate_summary(text)
with open("summary_ua.txt", "w", encoding="utf-8") as file:
    file.write(summary)


# Висновки

1. [Вихідний текст](./text_ua.txt)
2. [Анотація](./summary_ua.txt)

In [7]:
print(f"Довжина вихідного тексту: {len(text)}")
print(f"Довжина анотації: {len(summary)}")


Довжина вихідного тексту: 3110
Довжина анотації: 580


Таким чином текст довжиною `3110` слів зменшився до `580` слів.